# Libraries

In [3]:
from utils import *
from pprint import pprint
import pandas as pd
from spellchecker import SpellChecker
spell = SpellChecker()

import pickle

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

config = get_config('config.yaml')

# Read Input

In [2]:
df = pd.read_csv(config['csv_input_local']['all_reviews'], index_col=0)
df = df.reset_index(drop=True)
total_reviews = len(df)
unique_users  = len(df['userName'].unique())
mean = df['rating'].mean()

print(f'Total English reviews: {total_reviews} \n')
print(f'Total unique users : {unique_users}')
print(f'Total users who gave multiple reviews: {total_reviews - unique_users}\n')
print(f'Average rating for this app based on the textual reviews: {round(mean,2)} \n')

Total English reviews: 182361 

Total unique users : 171058
Total users who gave multiple reviews: 11303

Average rating for this app based on the textual reviews: 3.64 



# Preprocessing Input Data

## Remove Non-English User Reviews

In [ ]:
%%time
listOfNonEnglishIndex = []

for i in range(0, len(df)):
    reviewText = df['review'][i]
    
#     # for debugging purpose
#     print(reviewText)
#     print(isEnglishReview(reviewText))
#     print('\n')
    
    isEnglish, listToStr, english_score = isEnglishReview(reviewText)
    if isEnglish == False:
        listOfNonEnglishIndex.append(i)

In [ ]:
%%time
df_p1 = df.drop(df.index[listOfNonEnglishIndex])
total_reviews_before = len(df)
total_reviews_after = len(df_p1)
total_non_english_reviews = len(listOfNonEnglishIndex)

print(f'Total reviews (BEFORE): {total_reviews_before} \n')
print(f'Total reviews (AFTER): {total_reviews_after} \n')
print(f'Total Non-English reviews: {total_non_english_reviews} \n')

In [ ]:
df_p1.to_csv(config['csv_input_local']['all_reviews_p1'])

## Filtering Out Inconsistent User Review

- Because of an issue in SentiStrength path setting for Windows 10, I can't run it here
- Will able to run it via Linux or macOS

In [7]:
df_p2 = pd.read_csv(config['csv_input_local']['all_reviews_p2'], index_col=0)
df_p2 = df_p2.reset_index(drop=True)
df_p2

,userName,isEdited,review,rating,title,date
0,Kazem Sharan,False,I’ve been using bolt for a month now. I had a ...,1,Drivers are unprofessional,2019-09-20 09:25:04
1,19RN20,False,I’m so appalled at the attitude of the last dr...,1,Zero Stars.,2020-09-04 19:50:50
2,evolvingihsan,False,I’ve had many problems with the customer suppo...,1,HORRIBLE! HORRIBLE CUSTOMER SERVICE,2020-01-29 16:58:11
3,Sabcha08,False,So far majority of the drivers were very kind ...,3,Needs some updates,2019-07-05 12:17:37
4,Ellenyh79,False,I was super happy when Bolt launch as finally ...,1,Terrible customer service and grossly overchar...,2019-10-03 22:17:44
...,...,...,...,...,...,...
629,Vitaliy.l,False,Great app!,5,Great app,2016-03-22 15:29:49
630,cretchen,False,Great app!!,5,Great,2015-07-14 20:13:38
631,Nick_Name_Why,False,The drivers regularly get the wrong address du...,1,Poor location accuracy,2019-10-08 10:57:34
632,Illimar,False,Just love it!,5,Perfect UX,2017-06-23 20:34:04


## Filtering Out Uninformative Reviews

In [8]:
%run ./AR_Miner/AR_util.py
%run ./AR_Miner/AR_reviewInstance.py

# Inputs:
datasetName = "bolt" # four apps: facebook, templerun2, swiftkey, tapfish:
# datasetName = "templerun2" # four apps: facebook, templerun2, swiftkey, tapfish
rmStopWords = True # Removing stop words lead to information loss and bad f-score
rmRareWords = True # Remove the word with low frequency
skParse = False # set skParse True to directly read of the data that has been filtered out

# Outputs:
if(skParse == False):
    trainSet, testSet, unlabelSet, vocabulary = AR_parse(datasetName, rmStopWords, rmRareWords)

print('\n')

Vocabulary size for bolt : 1761
Training set Size: 110
Testing set Size: 522
Unlabeling set Size: 634




In [10]:
# for i in range(0, len(unlabelSet)):
#     print(unlabelSet[i].id)

In [11]:
%%time
%run ./AR_Miner/AR_classifier.py

useSVM = False # SVM is way better than EMNB in the testing
if(skParse == False):
    if(useSVM == False):
#         informRev, informMat = AR_emnb(trainSet, testSet, unlabelSet, vocabulary, datasetName)
        informRev, uninformRev, informMat = AR_emnb(trainSet, testSet, unlabelSet, vocabulary, datasetName)
    else:
#         informRev, informMat = AR_svm(trainSet, testSet, unlabelSet, vocabulary, datasetName)
        informRev, uninformRev, informMat = AR_svm(trainSet, testSet, unlabelSet, vocabulary, datasetName)
    # write the result back to the file (optional)
    # AR_writeReviews(informRev, datasetName)
    
else:
    # directly read from the file
    informRev, informMat, vocabulary = AR_loadReviews(datasetName)

print("Number of informative reviews: " + str(len(informRev)))
print("Number of uninformative reviews: " + str(len(uninformRev)))

No more progress, stopping EM at iteration 12
Average F-Score for the test data: 0.5037940063038874
Number of informative reviews: 625
Number of uninformative reviews: 9
Wall time: 3.36 s


In [23]:
df_p2.iloc[[366]]

,userName,isEdited,review,rating,title,date
366,Bug Reporter 1978,False,It’s spam on my phone. Don’t do it. How Apple ...,3,Stop Lock Screen Promos!,2019-12-19 11:45:37


In [29]:
listOfRemovedIndex_p3 = []

for i in range(0, len(uninformRev)):
#     print(uninformRev[i].id)
#     print(uninformRev[i].id - (len(trainSet) + len(testSet)))
#     uninformRev[i].printSelf()
#     print('\n')
    
    idxToRemove = uninformRev[i].id - (len(trainSet) + len(testSet))
    listOfRemovedIndex_p3.append(idxToRemove)

In [30]:
listOfRemovedIndex_p3

[276, 366, 389, 421, 461, 546, 559, 577, 589]

In [31]:
%%time
df_p3 = df_p2.drop(df_p2.index[listOfRemovedIndex_p3])
total_reviews_before = len(df_p2)
total_reviews_after = len(df_p3)
total_removed_reviews = len(listOfRemovedIndex_p3)

print(f'Total reviews (BEFORE): {total_reviews_before} \n')
print(f'Total reviews (AFTER): {total_reviews_after} \n')
print(f'Total removed reviews: {total_removed_reviews} \n')

Total reviews (BEFORE): 634 

Total reviews (AFTER): 625 

Total removed reviews: 9 

Wall time: 2.55 ms


In [33]:
# df_p3.to_csv(config['csv_input_local']['bolt_apple_p3'])

## Correcting Typos

In [42]:
%%time
cleaned_docs = remove_things(df_p3.review)
lists_of_words = list(sentences_to_words(cleaned_docs))
lists_of_words_no_stops = remove_stopwords(lists_of_words)
lists_of_words_no_stops

Wall time: 350 ms


[['using',
  'month',
  'good',
  'journeys',
  'horrendous',
  'ones',
  'recently',
  'first',
  'occasion',
  'told',
  'cross',
  'legs',
  'car',
  'might',
  'touch',
  'chair',
  'apparently',
  'encourages',
  'people',
  'sit',
  'still',
  'move',
  'car',
  'response',
  'got',
  'customer',
  'service',
  'refund',
  'second',
  'time',
  'dental',
  'appointment',
  'ordered',
  'cab',
  'driver',
  'came',
  'said',
  'going',
  'soon',
  'found',
  'location',
  'got',
  'frustrated',
  'swore',
  'said',
  'far',
  'man',
  'tired',
  'asked',
  'accepted',
  'first',
  'place',
  'want',
  'go',
  'far',
  'get',
  'proper',
  'explanation',
  'apology',
  'pain',
  'dental',
  'extraction',
  'cancel',
  'end',
  'said',
  'said',
  'would',
  'text',
  'let',
  'know',
  'messaged',
  'days',
  'ago',
  'still',
  'reply',
  'charged',
  'trip',
  'happen',
  'terrible',
  'terrible',
  'drivers',
  'bad',
  'customer',
  'service',
  'incidents',
  'like',
  'happen

In [49]:
list_of_typo_words = []
for sentence in lists_of_words_no_stops:
#     print(sentence)
    for word in sentence:
        if spell.correction(word) != word:
            print(word)
            print(spell.correction(word))
            list_of_typo_words.append(word)

xl
al
beckenham
beckham
cctv
cut
cctv
cut
deivers
drivers
fumingg
fuming
hru
thru
cornerr
corner
faq
far
kapten
kaiten
malta
malt
ffs
ifs
llocks
blocks
linkedin
linkedln
xx
ex
teip
trip
touran
toucan
tunis
tunes
gps
gas
viavan
vivian
ms
is
gps
gas
gps
gas
darkmode
darmody
usability
ability
gps
gas
viavan
vivian
postcodes
postcode
viavan
vivian
kapten
kaiten
gps
gas
slovakia
slovakian
kapten
kaiten
badd
bad
malta
malt
funke
funky
ms
is
funke
funky
meriza
media
londoner
londoners
btw
bow
gps
gas
аа
i
не
i
mr
my
masimba
marimba
pls
plus
befre
before
kapten
kaiten
xl
al
amaizing
amazing
kaunas
saunas
km
am
krakow
kraken
reaaly
really
latvia
latvian
ppl
pal
kapten
kaiten
gps
gas
gatwick
patrick
xs
is
xl
al
covid
couid
drivera
driver
kapten
kaiten
sh
so
ft
it
ft
it
ft
it
bmw
bow
tfl
til
sms
sums
unsubscribed
subscribed
uninstalled
installed
viavan
vivian
bonuss
bonus
nicee
nice
gps
gas
hrs
his
coudnt
count
happned
happened
muqtar
guitar
moalin
moulin
xl
al
xl
al
desaster
disaster
kaunas
saun

In [50]:
list_of_typo_words

['xl',
 'beckenham',
 'cctv',
 'cctv',
 'deivers',
 'fumingg',
 'hru',
 'cornerr',
 'faq',
 'kapten',
 'malta',
 'ffs',
 'llocks',
 'linkedin',
 'xx',
 'teip',
 'touran',
 'tunis',
 'gps',
 'viavan',
 'ms',
 'gps',
 'gps',
 'darkmode',
 'usability',
 'gps',
 'viavan',
 'postcodes',
 'viavan',
 'kapten',
 'gps',
 'slovakia',
 'kapten',
 'badd',
 'malta',
 'funke',
 'ms',
 'funke',
 'meriza',
 'londoner',
 'btw',
 'gps',
 'аа',
 'не',
 'mr',
 'masimba',
 'pls',
 'befre',
 'kapten',
 'xl',
 'amaizing',
 'kaunas',
 'km',
 'krakow',
 'reaaly',
 'latvia',
 'ppl',
 'kapten',
 'gps',
 'gatwick',
 'xs',
 'xl',
 'covid',
 'drivera',
 'kapten',
 'sh',
 'ft',
 'ft',
 'ft',
 'bmw',
 'tfl',
 'sms',
 'unsubscribed',
 'uninstalled',
 'viavan',
 'bonuss',
 'nicee',
 'gps',
 'hrs',
 'coudnt',
 'happned',
 'muqtar',
 'moalin',
 'xl',
 'xl',
 'desaster',
 'kaunas',
 'lyftt',
 'cs',
 'sms',
 'tallinn',
 'jdexgn',
 'cascais',
 'cascais',
 'mobey',
 'optionally',
 'cooll',
 'protus',
 'txfy',
 'huf',
 'huf',

## Processing Natural Language

In [56]:
df_p4 = df_p3.copy()
df_p4 = df_p4.reset_index(drop=True)
df_p4

,userName,isEdited,review,rating,title,date
0,Kazem Sharan,False,I’ve been using bolt for a month now. I had a ...,1,Drivers are unprofessional,2019-09-20 09:25:04
1,19RN20,False,I’m so appalled at the attitude of the last dr...,1,Zero Stars.,2020-09-04 19:50:50
2,evolvingihsan,False,I’ve had many problems with the customer suppo...,1,HORRIBLE! HORRIBLE CUSTOMER SERVICE,2020-01-29 16:58:11
3,Sabcha08,False,So far majority of the drivers were very kind ...,3,Needs some updates,2019-07-05 12:17:37
4,Ellenyh79,False,I was super happy when Bolt launch as finally ...,1,Terrible customer service and grossly overchar...,2019-10-03 22:17:44
...,...,...,...,...,...,...
620,Vitaliy.l,False,Great app!,5,Great app,2016-03-22 15:29:49
621,cretchen,False,Great app!!,5,Great,2015-07-14 20:13:38
622,Nick_Name_Why,False,The drivers regularly get the wrong address du...,1,Poor location accuracy,2019-10-08 10:57:34
623,Illimar,False,Just love it!,5,Perfect UX,2017-06-23 20:34:04


### Coreference Resolution

In [51]:
import logging;
logging.basicConfig(level=logging.INFO)
import neuralcoref

# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')
# nlp = spacy.load("en_core_web_sm")

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
doc = nlp(u'My sister has a dog. She loves him.')
print(doc._.coref_resolved)

My sister has a dog. My sister loves a dog.


In [60]:
df_p4.review[1]

'I’m so appalled at the attitude of the last driver I ordered from you. Three people recommended your services and I trusted those opinions. I’ve only taken four trips with Bolt and not ONE driver has worn a mask including the trips taken with my 7 month old baby. I ordered a XL car to help with moving a mirror and a bassinet to my destination, he took 15 minutes to get here, was on a call via loud speaker with a friend/family member. I asked if I could put my stuff into the boot as I was moving house and collecting the last items I had at my previous home. He declined and the person he was speaking to on the phone also got involved also. He then said it was my fault for ordering the wrong type of car, I told him I ordered the biggest car option on bolt. He said I would have to find another car and then told ME to cancel the drive. I refused so as he drove off he shouted something and left me, my baby and my things in the cold. Then to make matters worse, he cancelled the trip and put 

In [61]:
for i in range(0, len(df_p4)):
    textToCheck = df_p4.review[i]
    doc = nlp(textToCheck)
    hasCoref = doc._.has_coref
    
    print(textToCheck)
    print('\n')
    print(hasCoref)
    
    if(hasCoref == True):
        print(doc._.coref_resolved)
        
    print('\n')
    

I’ve been using bolt for a month now. I had a few good journeys but 2 horrendous ones recently. The first occasion I was told to not cross my legs in the car because it “might” touch his chair. So apparently bolt encourages people to sit still and not move in the car, which was the response I got from customer service and no refund. The second time, I had a dental appointment and ordered a cab, the driver came and then he said where are you going? As soon as he found out what the location was, he got frustrated and swore and said “this is too far man I’m tired”. I asked why he had accepted it in the first place if he didn’t want to go too far. I didn’t get any proper explanation and no apology. I was in pain because of my dental extraction too. So I had to cancel from my end because he said he couldn’t do it. He said he would text bolt and let them know. And I messaged Bolt 3 days ago and still no reply. They have charged me £3 for this trip that didn’t happen. Terrible terrible driver

I was super happy when Bolt launch as finally there’s fair competition for taxi apps in London. After using it a couple of times I was happy. Until I took a ride from Beckenham to Heathrow I was quoted an estimate of £45-50. But after the ride I noticed that Bolt had charged me £87 instead!!! I contacted customer service and they have been absolutely USELESS!! It’s been 2 weeks and I’m going round in circles with them asking me again what my route was, how much I was charged, sending them bank statements. They have refused to acknowledge the overcharge and not giving refund. If I had known they were going to charge such extortionate price I would have hired a local minicab company or gone with Uber instead! If I had raised such issue with Uber there would be an immediate refund. Watch out for these scammers!


True
I was super happy when Bolt launch as finally there’s fair competition for taxi apps in London. After using it a couple of times I was happy. Until I took a ride from Becken

So I’m trying out Bolt as a potential alternative to Uber, given the situation with Uber in London. Prices are good but Bolt needs to do a bit of work on the app!
1. The idea of being able to modify the pick up location should be added. You have to cancel and start agin even just for crossing the road
2. Being able to track the potential route would be useful, especially if you’re familiar with the route and can suggest modifications
Still - great to have an alternative to Uber! And drivers are all good too so far


True
So I’m trying out Bolt as a potential alternative to Uber, given the situation with Uber in London. Prices are good but Bolt needs to do a bit of work on the app!
1. The idea of being able to modify the pick up location should be added. You have to cancel and start agin even just for crossing the road
2. Being able to track the potential route would be useful, especially if you’re familiar with the route and can suggest modifications
Still - great to have an alternativ

Used three times so far after good friend recommended and send me invite that gave me good welcome discount which was plus! Overall professional drivers and friendly drivers. New or very decent cars and very quick pick up which was very great first impression. Keep up the good work!


False


Having used Uber for years and made over 1000 trips, I decided to try Bolt.
Cost wise it was extremely heap the first few rides, but now it is almost always more than Uber for the same route.
The app is very inaccurate and doesn’t recognise places of interest at all. I always have to message the driver to explain exactly where I am.
The app could definitely benefit from more development on the location accuracy.


True
Having used Uber for years and made over 1000 trips, I decided to try Bolt.
Cost wise it was extremely heap the first few rides, but now it is almost always more than Uber for the same route.
The app is very inaccurate and doesn’t recognise places of interest at all. I always have t

Nice service. Getting promotions from time to time. But the only thing that is lacking - two address request. It’s very common service. Instead of minimize communication with drivers, we need to explain that we would change final destination when we come closer to first one. So that driver won’t take next trip. Confusing.


True
Nice service. Getting promotions from time to time. But the only thing that is lacking - two address request. the only thing that is lacking - two address request’s very common service. Instead of minimize communication with drivers, we need to explain that we would change final destination when we come closer to first one. So that driver won’t take next trip. Confusing.


First trip was ok, second time I booked a trip, the app told me 8 min waiting time, the driver called me and said he would be 15 mins not 8 mins and did I want to cancel, I said no I would wait, after 15 mins waiting the driver cancelled the trip, after that I couldn’t not get another bolt ri

...

It’s good application.i have been used this app in Kaunas,Lithuania for 2-3 months .some of the driver can speak English and most of them are not speak English.some driver always took a longer route even it shows up on their equipment.i try to explain to them but seem like they don’t care and end up my payment was higher than expected.

I tried to contract bolt application service but it’s so difficult ,the only way for you to contact them is to email them.
No live chat 

the good things that you can shared your journey with another person.


True
It’s good application.i have been used this app in Kaunas,Lithuania for 2-3 months .some of the driver can speak English and most of them are not speak English.some driver always took a longer route even a longer route shows up on them equipment.i try to explain to them but seem like them don’t care and end up my payment was higher than expected.

I tried to contract bolt application service but a longer route’s so difficult ,the only way for

App is terrible. In the login process I'm supposed to receive SMS with login code. I've tried to enter my phone number for a number of times and nothing. I'm not receiving message with login code. Also, even worse... I needed a ride when it was heavy rain outside. Terrible service.


True
App is terrible. In the login process I'm supposed to receive SMS with login code. I've tried to enter my phone number for a number of times and nothing. I'm not receiving message with login code. Also, even worse... I needed a ride when a ride was heavy rain outside. Terrible service.


Worst taxi app in world. I got called dirty words so many times, harassed sexually and now they started to charge my card without even using it. I am getting transactions with “Tallinn/Bolt.eu” and I didn’t use this crap for several days. THIS IS PURE SCAM and must be removed from iTunes


True
Worst taxi app in world. I got called dirty words so many times, harassed sexually and now they started to charge my card wit

I took a ride in for 4070 HUF. The company charged me 407000 HUF. I can’t believe that this was mistake!! I was surprised to see -1333 EUR when checked my bank account. The support not answering to the messages. Not possible to join anyone by phone. The drivers are taking longer routes to charge you more. By far the worst taxi app you can imagine !!! Use Uber and you’d save yourself a lot of trouble. 0 stars for Taxify in every way.
#Update 28/12: Taxify blocked my number after this review. Well done because I don’t want  to hear about you anymore and that confirms the unprofessional methods of this company !!


True
I took a ride in for 4070 HUF. 4070 HUF charged me 407000 HUF. I can’t believe that this was mistake!! I was surprised to see -1333 EUR when checked my bank account. The support not answering to the messages. Not possible to join anyone by phone. The drivers are taking longer routes to charge you more. By far the worst taxi app you can imagine !!! Use Uber and you’d save y

Just signed up, they charged me with 2 euros and the bike didn’t move at all. Maybe the device was broken, I don’t know. But I didn’t use the bike and I had to pay for a trip.


True
Just signed up, they charged me with 2 euros and the bike didn’t move at all. Maybe the device was broken, I don’t know. But I didn’t use the bike and I had to pay for a trip.


BOLT is the worst ride sharing app I have ever tried. It took my money twice and I’ve never had a trip yet, this is unbelievable and has to be stopped. They are thieves and have to be removed from the market.


True
BOLT is the worst ride sharing app I have ever tried. app took my money twice and I’ve never had a trip yet, this is unbelievable and has to be stopped. They are thieves and have to be removed from the market.


It took more than 20 minutes for our ride to get to our destination, once it was close, it stated that it was finishing another ride and was going to take another 10 minutes. After I cancel the ride the app char



True
A very helpful app, but why I can't just watch the list of taxi companies with taxi companies phonenumbers?


Its fine.


False


BEST app ever! So easy to use and very functional!


False


Great app! Works perfectly.


False


Shows 1 min. Taxi arrives in 9min. Shows 3 min. Taxi arrives in 6 min etc.


True
Shows 1 min. Taxi arrives in 9min. Shows 3 min. Taxi arrives in 6 min etc.


Good app. I suggest


False


Very good and helpful app!


False


Great app!


False


Superb app! Thanks devs!!


False


Great app!


False


Great app!!


False


The drivers regularly get the wrong address due to some poor auto-replacement-with-the-local-hotspot function even though to the customer the app displays their correct address. Which leads the customer to blame the unsuspecting driver who can’t do anything about it. Bolt, your customer experience sucks!


True
The drivers regularly get the wrong address due to some poor auto-replacement-with-the-local-hotspot function even though to 

### Sentence Annotation

In [ ]:
# TODO

## Building Corpus

In [62]:
df_p5 = df_p4.copy()
df_p5

,userName,isEdited,review,rating,title,date
0,Kazem Sharan,False,I’ve been using bolt for a month now. I had a ...,1,Drivers are unprofessional,2019-09-20 09:25:04
1,19RN20,False,I’m so appalled at the attitude of the last dr...,1,Zero Stars.,2020-09-04 19:50:50
2,evolvingihsan,False,I’ve had many problems with the customer suppo...,1,HORRIBLE! HORRIBLE CUSTOMER SERVICE,2020-01-29 16:58:11
3,Sabcha08,False,So far majority of the drivers were very kind ...,3,Needs some updates,2019-07-05 12:17:37
4,Ellenyh79,False,I was super happy when Bolt launch as finally ...,1,Terrible customer service and grossly overchar...,2019-10-03 22:17:44
...,...,...,...,...,...,...
620,Vitaliy.l,False,Great app!,5,Great app,2016-03-22 15:29:49
621,cretchen,False,Great app!!,5,Great,2015-07-14 20:13:38
622,Nick_Name_Why,False,The drivers regularly get the wrong address du...,1,Poor location accuracy,2019-10-08 10:57:34
623,Illimar,False,Just love it!,5,Perfect UX,2017-06-23 20:34:04


In [64]:
cleaned_docs = remove_things(df_p5.review)
lists_of_words = list(sentences_to_words(cleaned_docs))
lists_of_words_no_stops = remove_stopwords(lists_of_words)

ngrams = make_bigrams(lists_of_words_no_stops)
# ngrams = make_trigrams(lists_of_words_no_stops)

data_lemmatized = lemmatize(ngrams, allowed_postags=['NOUN'])

Making bigrams...
Lemmatizing...


In [65]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
term_doc = [id2word.doc2bow(text) for text in texts]

# View
print(term_doc[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]]


In [67]:
tf_idf = models.TfidfModel(term_doc, smartirs='ntc')[term_doc]
[[(id2word[id], freq) for id, freq in cp] for cp in tf_idf[:1]]

[[('apology', 0.258303175195304),
  ('appointment', 0.19374341386795016),
  ('car', 0.1660609726432199),
  ('chair', 0.258303175195304),
  ('cross', 0.2304987993967995),
  ('day', 0.12187014507285714),
  ('driver', 0.07236556805706493),
  ('explanation', 0.2304987993967995),
  ('extraction', 0.258303175195304),
  ('incident', 0.2142342821987155),
  ('journey', 0.12323004598478411),
  ('leg', 0.258303175195304),
  ('location', 0.1423610134036225),
  ('man', 0.19374341386795016),
  ('month', 0.19374341386795016),
  ('occasion', 0.20269442359829498),
  ('one', 0.18642990640021098),
  ('pain', 0.258303175195304),
  ('people', 0.15244204814412854),
  ('place', 0.18024642394263304),
  ('refund', 0.14019220224671175),
  ('response', 0.15244204814412854),
  ('swore', 0.258303175195304),
  ('text', 0.258303175195304),
  ('trip', 0.10301677900469748)]]

In [73]:
with open(config['csv_input_local']['bolt_apple_corpus'], 'wb') as f:
    pickle.dump(tf_idf, f)